<a href="https://colab.research.google.com/github/mancher07/samsung/blob/main/GradioLive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.38.2 tiktoken torch numpy gradio

In [ ]:
import argparse
import os
import gradio as gr
import json
from threading import Thread
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [ ]:

MAX_LENGTH = 4096
DEFAULT_MAX_NEW_TOKENS = 1024

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b", torch_dtype=torch.bfloat16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def predict(message, history, system_prompt, temperature, max_tokens):
    instruction = system_prompt
    for human, assistant in history:
        instruction += f'user\n{human}\n\nassistant\n{assistant}\n'
    instruction += f'\nuser\n{message}\n\nassistant\n'

    problem = [instruction]
    stop_tokens = ["", ""]
    streamer = TextIteratorStreamer(tokenizer, timeout=100.0, skip_prompt=True, skip_special_tokens=True)
    enc = tokenizer(problem, return_tensors="pt", padding=True, truncation=True)
    input_ids = enc.input_ids
    attention_mask = enc.attention_mask

    if input_ids.shape[1] > MAX_LENGTH:
        input_ids = input_ids[:, -MAX_LENGTH:]

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    generate_kwargs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "streamer": streamer,
        "do_sample": True,
        "top_p": 0.95,
        "temperature": temperature,
        "max_new_tokens": max_tokens,
    }
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        if text in stop_tokens:
            break
        yield "".join(outputs)

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

# Variables globales y configuración del modelo
MAX_LENGTH = 4096
DEFAULT_MAX_NEW_TOKENS = 1024

# Cargar el modelo y el tokenizador
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b", torch_dtype=torch.bfloat16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def predict(message, history, system_prompt, temperature, max_tokens):
    instruction = system_prompt
    for human, assistant in history:
        instruction += f'user\n{human}\n\nassistant\n{assistant}\n'
    instruction += f'\nuser\n{message}\n\nassistant\n'

    problem = [instruction]
    streamer = TextIteratorStreamer(tokenizer, timeout=100.0, skip_prompt=True, skip_special_tokens=True)
    enc = tokenizer(problem, return_tensors="pt", padding=True, truncation=True)
    input_ids = enc.input_ids
    attention_mask = enc.attention_mask

    if input_ids.shape[1] > MAX_LENGTH:
        input_ids = input_ids[:, -MAX_LENGTH:]

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    generate_kwargs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "streamer": streamer,
        "do_sample": True,
        "top_p": 0.95,
        "temperature": temperature,
        "max_new_tokens": max_tokens,
    }

    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        yield "".join(outputs)
    yield "".join(outputs)

system_prompt = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."

interface = gr.ChatInterface(
    fn=predict,
    title="Stable Code Instruct Chat - Demo",
    description="Chat Model Stable Code 3B",
    theme="soft",
    chatbot=gr.Chatbot(label="Chat History"),
    textbox=gr.Textbox(placeholder="input", container=False, scale=7),
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
    additional_inputs=[
        gr.Textbox(system_prompt, label="System Prompt"),
        gr.Slider(0, 1, 0.9, label="Temperature"),
        gr.Slider(100, 2048, 1024, label="Max Tokens"),
    ],
    additional_inputs_accordion="Parameters",
)

interface.launch(debug=True, share=True)

time.sleep(5)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0557d5b2ef6fdaa144.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://0557d5b2ef6fdaa144.gradio.live
